## JSON to CSV transformation

### + pandas custom layer

In [ ]:
import json
import boto3
import DateTime
import pandas as pd
import time
from io import StringIO

s3 = boto3.client("s3")


def lambda_handler(event, context):
    
# specifying endpoint for the data import 

    bucket = "redditapibucket2"
    timestr = time.strftime("%Y%m%d")
    filename = "redditapi" + timestr + ".json"
    
# importing and reading JSON object 
    
    obj = s3.get_object(Bucket = bucket, Key = filename)
    
    content = obj["Body"]
    
    df = pd.read_json(content)
    
    df = df.T
    
    df = df[["id","title", "num_comments", "score", "timestamp", "url", "subreddit"]]
    
# transforming DataFrame timestamp to proper format

    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
    #print(df.head())

# specifying endpoint for the data export
    
    csvs3 = boto3.resource('s3')
    
# export to s3

    csvbucket = "redditbucketcsv" 
    csvfilename = "redditapi" + timestr + ".csv"
    
# save it into a buffer

    csv_buffer = StringIO()
    df.to_csv(csv_buffer)
    
    csvs3.Object(csvbucket,csvfilename).put(Body=csv_buffer.getvalue())

# print operation results

    print("put complete")
    